In [1]:
from BlackBox.lmart.LambdaMart_utils import LMARTGridsearch
import pandas as pd
import numpy as np
%reload_ext autoreload
%autoreload 2

In [2]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30, 40],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)
gridsearch_parameters = dict( # unbalanced sources
    train="../../outputs/scores/scores_tr.csv",
    valid="../../outputs/scores/scores_vl.csv",
    test="../../outputs/scores/scores_ts.csv",
    nDCG_at=15
)
gridsearch_parameters2 = dict( # balanced sources
    train="../../outputs/bal_scores/scores_tr.csv",
    valid="../../outputs/bal_scores/scores_vl.csv",
    test="../../outputs/bal_scores/scores_ts.csv",
    nDCG_at=15
)

# Grid-search

In [3]:
gs = LMARTGridsearch(**gridsearch_parameters)

In [4]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"LGBMRanker_unbalanced")

100%|██████████| 1440/1440 [05:12<00:00,  4.61it/s, nDCG_15=0.847]


# Metric evaluation

In [4]:
best_model = gs.load_model("./saved_models/LGBMRanker_unbalanced")

In [5]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,0.9867,0.9245,0.9098
Validation,0.9183,0.8598,0.8470
Test,0.8717,0.8520,0.8415


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.457,0.5693,0.5869
perfect_nDCG,1.000,1.0000,1.0000
worste_nDCG,0.125,0.3066,0.3341


# Grid search - balanced

In [9]:
gs = LMARTGridsearch(**gridsearch_parameters2)

In [10]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"LGBMRanker_balanced")

100%|██████████| 1440/1440 [05:01<00:00,  4.77it/s, nDCG_15=0.877]


In [11]:
best_model = gs.load_model("./saved_models/LGBMRanker_balanced")

In [12]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,0.9933,0.9596,0.9491
Validation,0.9600,0.8961,0.8772
Test,0.9250,0.9057,0.8909


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.4815,0.6017,0.6220
perfect_nDCG,1.0000,1.0000,1.0000
worste_nDCG,0.1382,0.3228,0.3545
